## current

In [ ]:

#=== 0. Drive 연결 / 필요 라이브러리 버전 점검 / 메시지 규정 지정 / 실행경로 지정
#  아마도 CLI 할 때 집중 점검 필요
print("🔹 Step 0/8: Installing minimal dependencies...")

# 0.1 GDrive Mount
from google.colab import drive
drive.mount('/content/drive')

# 0.2 Dependency check

!pip -q install nbformat==5.10.4 pandas==2.2.2

# 0.3 Turn off Deprecation and Future warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 0.4 Set working directory
import sys, os

# PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks"
PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks/autograder/src"
os.chdir(PKG_PARENT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔹 Step 0/8: Installing minimal dependencies...


In [ ]:
# ====== 1) 라이브러리 임포트 & 공통 유틸 ======
print("🔹 Step 1/8: Importing libraries & initializing...")

# 1.1 import common lib
import os, re, unicodedata
import nbformat, pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt


# 1.2 import autograder modules
# 1) policy
from autograder.policy import (
    BASE_SCORE, PENALTY_REQUIRED_MISS, PENALTY_REQUIRED_MISMATCH, PENALTY_OPTIONAL_MISS,
    outputs_equal, decide_status_and_match
)

# 2) io_utils
import nbformat
from autograder.io_utils import KST, now_kst, extract_id_and_name, _mtime_kst_str, _filesize_bytes, load_config

# 3) nb_utils
from autograder.nb_utils import ( LABEL_PATTERN, _normalize_code,_extract_label,
    _label_map, _nb_fingerprint, _cell_output_text, _indexes_with_labels, _sim,
    _label_key_robust
)

#4) report
from autograder.report import build_stats_block, build_excluded_summary_line

#5) label_tagging
from autograder.label_tagging import template_label_tagging

In [ ]:
# 1.3 Set path by each session and instance mode

session = load_config ("autograder/configs/sessions.toml", 9, "DEV")   # 일단 9차시 2024본
#session = load_config ("autograder/configs/sessions.toml", 9, "PROD")   # 일단 9차시 2025본

TEMPLATE_PATH = session["template_path"] #템플릿
SOLUTION_PATH = session["solution_path"] #정답
SUBMIT_DIR    = session["submit_dir"]   #제출파일들
OUT_DIR       = session["out_dir"]   #Autograde

# Folder for each excution
RUN_TS = now_kst().strftime("%Y%m%d_%H%M%S")
OUT = Path(OUT_DIR); OUT.mkdir(parents=True, exist_ok=True)
EXEC_DIR = OUT / "executed" / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)  # Autograde_MMDD_HHMM

# Tagged and audit file path
os.makedirs(OUT_DIR, exist_ok=True)
TAGGED_TEMP_PATH = str(Path(OUT_DIR) / "tagged_template.ipynb")
TAG_AUDIT_PATH = str(Path(OUT_DIR) / "tag_audit.csv")


In [ ]:
##deleted
#policy.py 분리
#io_utils.py 분리
#nb_utils.py 분리
#reports.py 분리
# sessions.toml 분리
# label_tagging.py 분리

## comment out
# logging.py








# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, sol_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    solcell = sol_map.get(lab, {}).get("cell")
    sol_out = _cell_output_text(solcell) if solcell else ""
    expected_output = bool(sol_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

# ====== 4) 템플릿 라벨 기반 태깅(없으면 생성) ======
#STRICT_REQUIRED_FROM_SOLUTION = True  # 정답에 코드가 있으면 무조건 필수로 태깅
# MIN_CODE_LEN = 5                      # "실코드" 최소 길이 기준

print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(TAGGED_TEMP_PATH):
    template_label_tagging(TEMPLATE_PATH, SOLUTION_PATH,
                           TAGGED_TEMP_PATH, TAG_AUDIT_PATH)
else :
    print ("Tagged template arleady exist!")


# ====== 5) 태그 읽기(라벨 기반) + 레퍼런스 NB 로드 ======
print("🔹 Step 3/8: Reading tagged template and solution (label-based)...")
## tmpl = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
## sol  = nbformat.read(SOLUTION_PATH, as_version=4)  # 출력 비교용
tmpl = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
sol  = nbformat.read(SOLUTION_PATH, as_version=4)

req_labels, opt_labels = set(), set()
req_idx, opt_idx = set(), set()

for i, c in enumerate(tmpl.cells):
    if c.cell_type != "code":
        continue
    tg = set(c.get("metadata", {}).get("tags", []))
    lab = _extract_label(c.source or "")
    if "required" in tg and lab:
        req_labels.add(lab); req_idx.add(i)
    if "optional_ex" in tg and lab:
        opt_labels.add(lab); opt_idx.add(i)

tmpl_lmap = _label_map(tmpl)
sol_lmap  = _label_map(sol)
REQ_INDEX_LABELS = _indexes_with_labels(tmpl, req_idx)
OPT_INDEX_LABELS = _indexes_with_labels(tmpl, opt_idx)
template_fp = _nb_fingerprint(tmpl)

# ====== 6) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 7) 제출물 처리 ======
print("🔹 Step 5/8: Collecting and grading submissions...")
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]

for path in paths:
    P=Path(path)
    if P.name in {Path(TEMPLATE_PATH).name, Path(SOLUTION_PATH).name, Path(TAGGED_TEMP_PATH).name}:
        continue

    sid, name = extract_id_and_name(P)
    sid2name[sid] = name
    sid2path[sid] = path

    # 노트북 로드
    try:
        nb = nbformat.read(path, as_version=4)
    except Exception as e:
        rows.append([sid, name, P.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=P.name
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, P.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    sol_lmap = _label_map(sol)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        solcell = sol_lmap.get(lab, {}).get("cell")
        sol_out = _cell_output_text(solcell) if solcell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        if not outputs_equal(stu_out, sol_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산

    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)

    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, P.name, score, status, reason, output_match, feedback])

    try:
        if mtime_kst(path).date() == now_kst().date():
            today_rows_tmp.append([sid, name, P.name, score, status, reason, output_match, feedback])
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)

# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 8) 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성

EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)

summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status","reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")


##202511031314

# 완료 메시지 + 로그
print("🔹 Step 8/8: Logging summary...")
kst_now_str = now_kst().strftime("%Y-%m-%d %H:%M:%S KST")
total_cnt = len(df)
prev_ids = prev_ids or set()
new_ids = set(df["student_id"].astype(str)) - prev_ids if prev_ids else set(df["student_id"].astype(str))
new_cnt = len(new_ids)

# 전체 채점 제외 라벨(정답 출력 없음) 한 줄 요약
excluded_req_str = ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) if EXCLUDED_REQ_ALL else "없음"
excluded_opt_str = ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) if EXCLUDED_OPT_ALL else "없음"

## 점수 통계 및 분포 산출
STATS_BLOCK = build_stats_block(df)

CONFIG = {
    "RUN_TS": RUN_TS,
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "SOLUTION_PATH": str(SOLUTION_PATH),
    "SUBMIT_DIR": str(SUBMIT_DIR),
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tmpl, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tmpl, opt_idx),
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",
    "TIMEZONE": "KST (UTC+9)",
}

log_lines = [
    f"=== Autograde Run @ {kst_now_str} ({CONFIG['RUN_TS']}) ===",
    f"OUT_DIR: {CONFIG['OUT_DIR']}",
    f"EXEC_DIR: {CONFIG['EXEC_DIR']}",
    f"SUMMARY_FILE_LATEST: {CONFIG['SUMMARY_FILE_LATEST']}",
    f"SIMILAR_FILE_LATEST: {CONFIG['SIMILAR_FILE_LATEST']}",
    f"NEWTODAY_FILE_LATEST: {CONFIG['NEWTODAY_FILE_LATEST']}",
    f"SUBMIT_DIR: {CONFIG['SUBMIT_DIR']}",
    f"TEMPLATE: {CONFIG['TEMPLATE_PATH']}",
    f"SOLUTION: {CONFIG['SOLUTION_PATH']}",
    f"TOTAL_STUDENTS: {total_cnt}",
    f"NEWLY_GRADED: {new_cnt}",
    f"TODAY_FILES(KST {now_kst().date()}): {len(df_today)}",
    f"TAG_AUDIT_LATEST: {'tag_audit_latest.csv'}",
    f"REQUIRED_CELLS: {CONFIG['REQUIRED_CELL_COUNT']}",
    f"OPTIONAL_CELLS: {CONFIG['OPTIONAL_CELL_COUNT']}",
    f"REQUIRED_CELL_INDEXES: {CONFIG['REQUIRED_CELL_INDEXES']}",
    f"OPTIONAL_CELL_INDEXES: {CONFIG['OPTIONAL_CELL_INDEXES']}",
    f"REQUIRED_CELL_MAP: {CONFIG['REQUIRED_CELL_MAP']}",
    f"OPTIONAL_CELL_MAP: {CONFIG['OPTIONAL_CELL_MAP']}",
    f"SCORE_RULE: {CONFIG['SCORE_RULE']}",
    f"TIMEZONE: {CONFIG['TIMEZONE']}",
    f"채점 제외(정답 출력 없음): 필수=[{excluded_req_str}], 연습=[{excluded_opt_str}]"
]

# 신규 ID
if new_cnt > 0:
    log_lines.append("NEW_IDS: " + ", ".join(sorted(new_ids)))

# 점수 통계 & 분포를 로그에 포함
log_lines += STATS_BLOCK

log_msg = "\n".join(log_lines) + "\n"
log_path = Path(OUT_DIR) / "autograde_run.log"
with open(log_path, "a", encoding="utf-8") as f:
    f.write(log_msg)

print("🗒️ Log appended to:", str(log_path))

# 화면 출력용도 함께 표시

print(
    "✅ 완료:", OUT_DIR,
    "\n- 실행시각:", kst_now_str,
    f"\n- 전체 채점 학생 수: {total_cnt}명",
    f"\n- 새롭게 추가 채점한 학생 수: {new_cnt}명",
    f"\n- 오늘 들어온 파일 수(KST {now_kst().date()}): {len(df_today)}건",
    "\n- 최신 요약:", Path(summary_latest).name,
    f"\n- 최신 유사도: {Path(similar_latest).name if pairs else 'N/A'}",
    "\n- 최신 Today:", Path(newtoday_latest).name,
    f"\n- 실행 산출물 폴더: executed/{RUN_TS}/",
    "\n- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트)",
    "\n".join(["\n📊 Score & Distribution Summary"] + STATS_BLOCK))


🔹 Step 1/8: Importing libraries & initializing...
🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
Tagged template arleady exist!
🔹 Step 3/8: Reading tagged template and solution (label-based)...
🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...
🔹 Step 5/8: Collecting and grading submissions...
🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...
🔹 Step 7/8: Saving outputs...
🔹 Step 8/8: Logging summary...
🗒️ Log appended to: /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시/autograde_run.log
✅ 완료: /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시 
- 실행시각: 2025-11-04 13:03:46 KST 
- 전체 채점 학생 수: 112명 
- 새롭게 추가 채점한 학생 수: 0명 
- 오늘 들어온 파일 수(KST 2025-11-04): 0건 
- 최신 요약: summary_static_with_name_latest.csv 
- 최신 유사도: similar_pairs_latest.csv 
- 최신 Today: new_today_latest.csv 
- 실행 산출물 폴더: executed/20251104_130322/ 
- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트) 
📊 Score & Distribution Summary
SCORE STATS:
- mean=95.4, m

## archive

### 2025 1104 1832

In [ ]:

# ====== 2) 템플릿 레이블 태깅 (필수/옵션 ) ======
print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(TAGGED_TEMP_PATH):
    template_label_tagging(TEMPLATE_PATH, ANSWER_PATH,
                           TAGGED_TEMP_PATH, TAG_AUDIT_PATH)
    print("Tagged template created!")
else :
    print ("Tagged template arleady exist!")


# ====== 3) 태그 템플릿으로부터 필수/연습 레이블 정보 획득  ======
print("🔹 Step 3/8: Reading tagged template and answer (label-based)...")
req_labels, opt_labels, req_idx, opt_idx, template_fp = \
    classify_labels(TAGGED_TEMP_PATH)


# ====== 4) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 5) 제출물 처리 (채점) ======
print("🔹 Step 5/8: Collecting and grading submissions...")

# 채점 관련 변수
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]



# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, ans_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    anscell = ans_map.get(lab, {}).get("cell")
    ans_out = _cell_output_text(anscell) if anscell else ""
    expected_output = bool(ans_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

tagged_nb = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
ans  = nbformat.read(ANSWER_PATH, as_version=4)

skip_names = {
    TEMPLATE_PATH.name,
    ANSWER_PATH.name,
    TAGGED_TEMP_PATH.name,
}

#  여기서 부터 채점
for p in map(Path, paths):
    if p.name in skip_names:
        continue

### for path in paths:
#    P=Path(path)
#    if P.name in {Path(TEMPLATE_PATH).name, Path(ANSWER_PATH).name, Path(TAGGED_TEMP_PATH).name}:
#        continue

    sid, name = extract_id_and_name(p)  #*#P
    sid2name[sid] = name
    sid2path[sid] = p   #*#P  path -> p

    # 노트북 로드
    try:
        nb = nbformat.read(p, as_version=4)  #*#P  path -> p
    except Exception as e:
        rows.append([sid, name, p.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"]) #*#P
        try:
            if mtime_kst(p).date() == now_kst().date():   #*#P  path -> p
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=p.name #*#P
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, p.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])  #*#P
        try:
            if mtime_kst(p).date() == now_kst().date():   #*#P  path -> p
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    ans_lmap = _label_map(ans)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        anscell = ans_lmap.get(lab, {}).get("cell")
        ans_out = _cell_output_text(anscell) if anscell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        if not outputs_equal(stu_out, ans_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산

    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)

    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, p.name, score, status, reason, output_match, feedback])   #*#P

    try:
        if mtime_kst(path).date() == now_kst().date():  #*#P  path -> p
            today_rows_tmp.append([sid, name, p.name, score, status, reason,
                                   output_match, feedback]) #*#P
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)


# ====== 6) 유사도 비교 ======
# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 7) 출력물 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성
"""
EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)
"""
summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

summary_df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status",
             "reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
summary_df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
summary_df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=summary_df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")


# ====== 8) 수행결과 요약 실행로그 생성 및 출력  ======
print("🔹 Step 8/8: Logging summary...")

# 8.1 추가 요약정보 산출
# 신규추가자 id 생성
prev_ids = prev_ids or set()
new_ids = set(summary_df["student_id"].astype(str)) - prev_ids \
           if prev_ids else set(summary_df["student_id"].astype(str))

## 점수 통계 및 분포 산출
STATS_BLOCK = build_stats_block(summary_df)

# 8.2 요약정보 정리
CONFIG = {
    # ── Run / Time Info ─────────────────────────────────────────────
    "TODAY_DATE": now_kst().date(),
    "KST_NOW": now_kst().strftime("%Y-%m-%d %H:%M:%S KST"),
    "TIMEZONE": "KST (UTC+9)",
    "RUN_TS": RUN_TS,

    # ── Directories (as str) ────────────────────────────────────────
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUBMIT_DIR": str(SUBMIT_DIR),

    # ── File basenames (name only) ──────────────────────────────────
    "TEMPLATE_FILE": Path(TEMPLATE_PATH).name,
    "ANSWER_FILE": Path(ANSWER_PATH).name,
    "TAGGED_TEMP_FILE": Path(TAGGED_TEMP_PATH).name,
    "TAG_AUDIT_FILE": Path(TAG_AUDIT_PATH).name,
    "RUN_LOG_FILE": "autograde_run.log",
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,


    # ── Full paths (as str) ─────────────────────────────────────────
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "ANSWER_PATH": str(ANSWER_PATH),
    "TAGGED_TEMP_PATH": str(TAGGED_TEMP_PATH),

    # ── Scoring / Thresholds ────────────────────────────────────────
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",

    # ── Required/Optional cells (counts / indexes / maps) ───────────
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tagged_nb, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tagged_nb, opt_idx),
    "EXCLUDED_REQ_ALL": ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) \
                          if EXCLUDED_REQ_ALL else "없음",
    "EXCLUDED_OPT_ALL": ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) \
                          if EXCLUDED_OPT_ALL else "없음",

    # ── Dataset stats ────────────────────────────────────────────────
    "TOTAL_CNT": len(summary_df),
    "NEW_CNT": len(new_ids),
    "TODAY_CNT": len(df_today),
}

# 8.3 실행 로그 파일 생성
# generate log format
log_lines = build_run_log_lines(CONFIG, STATS_BLOCK, new_ids=new_ids)

# save log file
log_path = Path(CONFIG["OUT_DIR"]) / "autograde_run.log"
log_path.parent.mkdir(parents=True, exist_ok=True)
with log_path.open("a", encoding="utf-8") as f:
    f.write("\n".join(log_lines) + "\n")

print("🗒️ Log appended to:", CONFIG['RUN_LOG_FILE'])

# 8.4 실행 결과 요약 출력
# print running summary to output terminal
run_summary = render_run_summary(CONFIG, STATS_BLOCK)
print(run_summary)


In [1]:

#=== 0. Drive 연결 / 필요 라이브러리 버전 점검 / 메시지 규정 지정 / 실행경로 지정  ===
#  아마도 CLI 할 때 집중 점검 필요
print("🔹 Step 0/8: Installing minimal dependencies...")

# 0.1 GDrive Mount
from google.colab import drive
drive.mount('/content/drive')

# 0.2 Dependency check

!pip -q install nbformat==5.10.4 pandas==2.2.2

# 0.3 Turn off Deprecation and Future warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 0.4 Set working directory
import sys, os

# PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks"
PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks/autograder/src"
os.chdir(PKG_PARENT)

🔹 Step 0/8: Installing minimal dependencies...


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# ====== 1) 라이브러리 임포트 & 공통 유틸 ======
print("🔹 Step 1/8: Importing libraries & initializing...")

# 1.1 import common lib
import os, re, unicodedata
import nbformat, pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt


# 1.2 import autograder modules
# 1) policy
from autograder.policy import (  BASE_SCORE, PENALTY_REQUIRED_MISS,
    PENALTY_REQUIRED_MISMATCH, PENALTY_OPTIONAL_MISS,
    outputs_equal, decide_status_and_match )

# 2) io_utils
from autograder.io_utils import (KST, now_kst, extract_id_and_name,
    _mtime_kst_str, _filesize_bytes, load_config)

# 3) nb_utils
from autograder.nb_utils import ( LABEL_PATTERN, _normalize_code,_extract_label,
    _label_map, _nb_fingerprint, _cell_output_text, _indexes_with_labels, _sim,
    _label_key_robust)


#4) report
from autograder.report import ( build_stats_block, build_excluded_summary_line,
            render_run_summary, build_run_log_lines)

#5) label_tagging
from autograder.label_tagging import (template_label_tagging, classify_labels)

In [ ]:
# 1.3 Set path by each session and instance mode

session = load_config ("autograder/configs/sessions.toml", 9, "DEV")   # 일단 9차시 2024본
#session = load_config ("autograder/configs/sessions.toml", 9, "PROD")   # 일단 9차시 2025본

TEMPLATE_PATH = Path(session["template_path"])  # 템플릿
ANSWER_PATH   = Path(session["answer_path"])    # 정답
SUBMIT_DIR    = Path(session["submit_dir"])     # 제출파일들
OUT_DIR       = Path(session["out_dir"])        # Autograde 출력 루트


# Tagged template and audit file path
TAGGED_TEMP_PATH = OUT_DIR / "tagged_template.ipynb"
TAG_AUDIT_PATH   = OUT_DIR / "tag_audit.csv"

# Folder for each excution output
RUN_TS   = now_kst().strftime("%Y%m%d_%H%M%S")
EXEC_DIR = OUT_DIR / "executed" / RUN_TS

OUT_DIR.mkdir(parents=True, exist_ok=True)
EXEC_DIR.mkdir(parents=True, exist_ok=True)




In [ ]:

# ====== 2) 템플릿 레이블 태깅 (필수/옵션 ) ======
print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(TAGGED_TEMP_PATH):
    template_label_tagging(TEMPLATE_PATH, ANSWER_PATH,
                           TAGGED_TEMP_PATH, TAG_AUDIT_PATH)
    print("Tagged template created!")
else :
    print ("Tagged template arleady exist!")


# ====== 3) 태그 템플릿으로부터 필수/연습 레이블 정보 획득  ======
print("🔹 Step 3/8: Reading tagged template and answer (label-based)...")
req_labels, opt_labels, req_idx, opt_idx, template_fp = \
    classify_labels(TAGGED_TEMP_PATH)


# ====== 4) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 5) 제출물 처리 (채점) ======
print("🔹 Step 5/8: Collecting and grading submissions...")

# 채점 관련 변수
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]



# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, ans_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    anscell = ans_map.get(lab, {}).get("cell")
    ans_out = _cell_output_text(anscell) if anscell else ""
    expected_output = bool(ans_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

tagged_nb = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
ans  = nbformat.read(ANSWER_PATH, as_version=4)

skip_names = {
    TEMPLATE_PATH.name,
    ANSWER_PATH.name,
    TAGGED_TEMP_PATH.name,
}

#  여기서 부터 채점
for p in map(Path, paths):
    if p.name in skip_names:
        continue

### for path in paths:
#    P=Path(path)
#    if P.name in {Path(TEMPLATE_PATH).name, Path(ANSWER_PATH).name, Path(TAGGED_TEMP_PATH).name}:
#        continue

    sid, name = extract_id_and_name(p)  #*#P
    sid2name[sid] = name
    sid2path[sid] = p   #*#P  path -> p

    # 노트북 로드
    try:
        nb = nbformat.read(p, as_version=4)  #*#P  path -> p
    except Exception as e:
        rows.append([sid, name, p.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"]) #*#P
        try:
            if mtime_kst(p).date() == now_kst().date():   #*#P  path -> p
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=p.name #*#P
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, p.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])  #*#P
        try:
            if mtime_kst(p).date() == now_kst().date():   #*#P  path -> p
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    ans_lmap = _label_map(ans)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        anscell = ans_lmap.get(lab, {}).get("cell")
        ans_out = _cell_output_text(anscell) if anscell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        if not outputs_equal(stu_out, ans_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산

    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)

    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, p.name, score, status, reason, output_match, feedback])   #*#P

    try:
        if mtime_kst(path).date() == now_kst().date():  #*#P  path -> p
            today_rows_tmp.append([sid, name, p.name, score, status, reason,
                                   output_match, feedback]) #*#P
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)


# ====== 6) 유사도 비교 ======
# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 7) 출력물 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성

EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)

summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status",
             "reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")


# ====== 8) 수행결과 요약 실행로그 생성 및 출력  ======
print("🔹 Step 8/8: Logging summary...")

# 8.1 추가 요약정보 산출
# 신규추가자 id 생성
prev_ids = prev_ids or set()
new_ids = set(df["student_id"].astype(str)) - prev_ids \
           if prev_ids else set(df["student_id"].astype(str))

## 점수 통계 및 분포 산출
STATS_BLOCK = build_stats_block(df)

# 8.2 요약정보 정리
CONFIG = {
    # ── Run / Time Info ─────────────────────────────────────────────
    "TODAY_DATE": now_kst().date(),
    "KST_NOW": now_kst().strftime("%Y-%m-%d %H:%M:%S KST"),
    "TIMEZONE": "KST (UTC+9)",
    "RUN_TS": RUN_TS,

    # ── Directories (as str) ────────────────────────────────────────
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUBMIT_DIR": str(SUBMIT_DIR),

    # ── File basenames (name only) ──────────────────────────────────
    "TEMPLATE_FILE": Path(TEMPLATE_PATH).name,
    "ANSWER_FILE": Path(ANSWER_PATH).name,
    "TAGGED_TEMP_FILE": Path(TAGGED_TEMP_PATH).name,
    "TAG_AUDIT_FILE": Path(TAG_AUDIT_PATH).name,
    "RUN_LOG_FILE": "autograde_run.log",
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,


    # ── Full paths (as str) ─────────────────────────────────────────
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "ANSWER_PATH": str(ANSWER_PATH),
    "TAGGED_TEMP_PATH": str(TAGGED_TEMP_PATH),

    # ── Scoring / Thresholds ────────────────────────────────────────
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",

    # ── Required/Optional cells (counts / indexes / maps) ───────────
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tagged_nb, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tagged_nb, opt_idx),
    "EXCLUDED_REQ_ALL": ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) \
                          if EXCLUDED_REQ_ALL else "없음",
    "EXCLUDED_OPT_ALL": ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) \
                          if EXCLUDED_OPT_ALL else "없음",

    # ── Dataset stats ────────────────────────────────────────────────
    "TOTAL_CNT": len(df),
    "NEW_CNT": len(new_ids),
    "TODAY_CNT": len(df_today),
}

# 8.3 실행 로그 파일 생성
# generate log format
log_lines = build_run_log_lines(CONFIG, STATS_BLOCK, new_ids=new_ids)

# save log file
log_path = Path(CONFIG["OUT_DIR"]) / "autograde_run.log"
log_path.parent.mkdir(parents=True, exist_ok=True)
with log_path.open("a", encoding="utf-8") as f:
    f.write("\n".join(log_lines) + "\n")

print("🗒️ Log appended to:", CONFIG['RUN_LOG_FILE'])

# 8.4 실행 결과 요약 출력
# print running summary to output terminal
run_summary = render_run_summary(CONFIG, STATS_BLOCK)
print(run_summary)


### 2025 1104 1500

In [ ]:

#=== 0. Drive 연결 / 필요 라이브러리 버전 점검 / 메시지 규정 지정 / 실행경로 지정  ===
#  아마도 CLI 할 때 집중 점검 필요
print("🔹 Step 0/8: Installing minimal dependencies...")

# 0.1 GDrive Mount
from google.colab import drive
drive.mount('/content/drive')

# 0.2 Dependency check

!pip -q install nbformat==5.10.4 pandas==2.2.2

# 0.3 Turn off Deprecation and Future warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 0.4 Set working directory
import sys, os

# PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks"
PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks/autograder/src"
os.chdir(PKG_PARENT)

🔹 Step 0/8: Installing minimal dependencies...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ====== 1) 라이브러리 임포트 & 공통 유틸 ======
print("🔹 Step 1/8: Importing libraries & initializing...")

# 1.1 import common lib
import os, re, unicodedata
import nbformat, pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt


# 1.2 import autograder modules
# 1) policy
from autograder.policy import (
    BASE_SCORE, PENALTY_REQUIRED_MISS, PENALTY_REQUIRED_MISMATCH, PENALTY_OPTIONAL_MISS,
    outputs_equal, decide_status_and_match
)

# 2) io_utils
import nbformat
from autograder.io_utils import KST, now_kst, extract_id_and_name, _mtime_kst_str, _filesize_bytes, load_config

# 3) nb_utils
from autograder.nb_utils import ( LABEL_PATTERN, _normalize_code,_extract_label,
    _label_map, _nb_fingerprint, _cell_output_text, _indexes_with_labels, _sim,
    _label_key_robust
)

#4) report
from autograder.report import build_stats_block, build_excluded_summary_line

#5) label_tagging
from autograder.label_tagging import template_label_tagging

🔹 Step 1/8: Importing libraries & initializing...


In [ ]:
# 1.3 Set path by each session and instance mode

session = load_config ("autograder/configs/sessions.toml", 9, "DEV")   # 일단 9차시 2024본
#session = load_config ("autograder/configs/sessions.toml", 9, "PROD")   # 일단 9차시 2025본

TEMPLATE_PATH = Path(session["template_path"])  # 템플릿
ANSWER_PATH   = Path(session["answer_path"])    # 정답
SUBMIT_DIR    = Path(session["submit_dir"])     # 제출파일들
OUT_DIR       = Path(session["out_dir"])        # Autograde 출력 루트


# Tagged template and audit file path
TAGGED_TEMP_PATH = OUT_DIR / "tagged_template.ipynb"
TAG_AUDIT_PATH   = OUT_DIR / "tag_audit.csv"

# Folder for each excution output
RUN_TS   = now_kst().strftime("%Y%m%d_%H%M%S")
EXEC_DIR = OUT_DIR / "executed" / RUN_TS

OUT_DIR.mkdir(parents=True, exist_ok=True)
EXEC_DIR.mkdir(parents=True, exist_ok=True)




In [ ]:
"""# 1.3 Set path by each session and instance mode

session = load_config ("autograder/configs/sessions.toml", 9, "DEV")   # 일단 9차시 2024본
#session = load_config ("autograder/configs/sessions.toml", 9, "PROD")   # 일단 9차시 2025본

TEMPLATE_PATH = session["template_path"] # 템플릿
ANSWER_PATH   = session["answer_path"]   # 정답
SUBMIT_DIR    = session["submit_dir"]    # 제출파일들
OUT_DIR       = session["out_dir"]       # Autograde

# Tagged template and audit file path
os.makedirs(OUT_DIR, exist_ok=True)
TAGGED_TEMP_PATH = str(Path(OUT_DIR) / "tagged_template.ipynb")
TAG_AUDIT_PATH = str(Path(OUT_DIR) / "tag_audit.csv")

# Folder for each excution output
RUN_TS = now_kst().strftime("%Y%m%d_%H%M%S")
OUT = Path(OUT_DIR); OUT.mkdir(parents=True, exist_ok=True)
EXEC_DIR = OUT / "executed" / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)  # Autograde_MMDD_HHMM"""




'# 1.3 Set path by each session and instance mode\n\nsession = load_config ("autograder/configs/sessions.toml", 9, "DEV")   # 일단 9차시 2024본\n#session = load_config ("autograder/configs/sessions.toml", 9, "PROD")   # 일단 9차시 2025본\n\nTEMPLATE_PATH = session["template_path"] # 템플릿\nANSWER_PATH   = session["answer_path"]   # 정답\nSUBMIT_DIR    = session["submit_dir"]    # 제출파일들\nOUT_DIR       = session["out_dir"]       # Autograde\n\n# Tagged template and audit file path\nos.makedirs(OUT_DIR, exist_ok=True)\nTAGGED_TEMP_PATH = str(Path(OUT_DIR) / "tagged_template.ipynb")\nTAG_AUDIT_PATH = str(Path(OUT_DIR) / "tag_audit.csv")\n\n# Folder for each excution output\nRUN_TS = now_kst().strftime("%Y%m%d_%H%M%S")\nOUT = Path(OUT_DIR); OUT.mkdir(parents=True, exist_ok=True)\nEXEC_DIR = OUT / "executed" / RUN_TS\nEXEC_DIR.mkdir(parents=True, exist_ok=True)  # Autograde_MMDD_HHMM'

In [ ]:
##deleted
#policy.py 분리
#io_utils.py 분리
#nb_utils.py 분리
#reports.py 분리
# sessions.toml 분리
# label_tagging.py 분리

## comment out
# logging.py

# ====== 2) 템플릿 레이블 태깅 (필수/옵션 ) ======
print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(TAGGED_TEMP_PATH):
    template_label_tagging(TEMPLATE_PATH, ANSWER_PATH,
                           TAGGED_TEMP_PATH, TAG_AUDIT_PATH)
else :
    print ("Tagged template arleady exist!")


# ====== 3) 태그 템플릿과 출력 레이블 비교 + 레퍼런스 NB 로드 ======
print("🔹 Step 3/8: Reading tagged template and answer (label-based)...")


def classify_labels(tagged_temp_path):
    tagged_nb = nbformat.read(tagged_temp_path, as_version=4)
    # ans_nb  = nbformat.read(answer_path, as_version=4)

    req_labels, opt_labels = set(), set()
    req_idx, opt_idx = set(), set()

    for i, c in enumerate(tagged_nb.cells):
        if c.cell_type != "code":
            continue
        tg = set(c.get("metadata", {}).get("tags", []))
        lab = _extract_label(c.source or "")
        if "required" in tg and lab:
            req_labels.add(lab); req_idx.add(i)
        if "optional_ex" in tg and lab:
            opt_labels.add(lab); opt_idx.add(i)


    template_fp = _nb_fingerprint(tagged_nb)
    return req_labels, opt_labels, req_idx, opt_idx, template_fp

req_labels, opt_labels, req_idx, opt_idx, template_fp = \
    classify_labels(TAGGED_TEMP_PATH)

"""    tmpl = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
    ans  = nbformat.read(ANSWER_PATH, as_version=4)

    req_labels, opt_labels = set(), set()
    req_idx, opt_idx = set(), set()

    for i, c in enumerate(tmpl.cells):
        if c.cell_type != "code":
            continue
        tg = set(c.get("metadata", {}).get("tags", []))
        lab = _extract_label(c.source or "")
        if "required" in tg and lab:
            req_labels.add(lab); req_idx.add(i)
        if "optional_ex" in tg and lab:
            opt_labels.add(lab); opt_idx.add(i)


    template_fp = _nb_fingerprint(tmpl)
"""




# ====== 4) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 5) 제출물 처리 (채점) ======
print("🔹 Step 5/8: Collecting and grading submissions...")

# 채점 관련 변수
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]



# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, ans_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    anscell = ans_map.get(lab, {}).get("cell")
    ans_out = _cell_output_text(anscell) if anscell else ""
    expected_output = bool(ans_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

tagged_nb = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
ans  = nbformat.read(ANSWER_PATH, as_version=4)

skip_names = {
    TEMPLATE_PATH.name,
    ANSWER_PATH.name,
    TAGGED_TEMP_PATH.name,
}

#  여기서 부터 채점
for p in map(Path, paths):
    if p.name in skip_names:
        continue

### for path in paths:
#    P=Path(path)
#    if P.name in {Path(TEMPLATE_PATH).name, Path(ANSWER_PATH).name, Path(TAGGED_TEMP_PATH).name}:
#        continue

    sid, name = extract_id_and_name(p)  #*#P
    sid2name[sid] = name
    sid2path[sid] = p   #*#P  path -> p

    # 노트북 로드
    try:
        nb = nbformat.read(p, as_version=4)  #*#P  path -> p
    except Exception as e:
        rows.append([sid, name, p.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"]) #*#P
        try:
            if mtime_kst(p).date() == now_kst().date():   #*#P  path -> p
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=p.name #*#P
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, p.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])  #*#P
        try:
            if mtime_kst(p).date() == now_kst().date():   #*#P  path -> p
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    ans_lmap = _label_map(ans)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        anscell = ans_lmap.get(lab, {}).get("cell")
        ans_out = _cell_output_text(anscell) if anscell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        if not outputs_equal(stu_out, ans_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산

    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)

    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, p.name, score, status, reason, output_match, feedback])   #*#P

    try:
        if mtime_kst(path).date() == now_kst().date():  #*#P  path -> p
            today_rows_tmp.append([sid, name, p.name, score, status, reason, output_match, feedback]) #*#P
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)


# ====== 6) 유사도 비교 ======
# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 7) 출력물 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성

EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)

summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status","reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")


##202511031314

# ====== 8) 완료 메시지 출력  ======
# 완료 메시지 + 로그
print("🔹 Step 8/8: Logging summary...")
kst_now_str = now_kst().strftime("%Y-%m-%d %H:%M:%S KST")
total_cnt = len(df)
prev_ids = prev_ids or set()
new_ids = set(df["student_id"].astype(str)) - prev_ids if prev_ids else set(df["student_id"].astype(str))
new_cnt = len(new_ids)

# 전체 채점 제외 라벨(정답 출력 없음) 한 줄 요약
excluded_req_str = ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) if EXCLUDED_REQ_ALL else "없음"
excluded_opt_str = ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) if EXCLUDED_OPT_ALL else "없음"

## 점수 통계 및 분포 산출
STATS_BLOCK = build_stats_block(df)

CONFIG = {
    "RUN_TS": RUN_TS,
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "ANSWER_PATH": str(ANSWER_PATH),
    "SUBMIT_DIR": str(SUBMIT_DIR),
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tagged_nb, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tagged_nb, opt_idx),
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",
    "TIMEZONE": "KST (UTC+9)",
}

log_lines = [
    f"=== Autograde Run @ {kst_now_str} ({CONFIG['RUN_TS']}) ===",
    f"OUT_DIR: {CONFIG['OUT_DIR']}",
    f"EXEC_DIR: {CONFIG['EXEC_DIR']}",
    f"SUMMARY_FILE_LATEST: {CONFIG['SUMMARY_FILE_LATEST']}",
    f"SIMILAR_FILE_LATEST: {CONFIG['SIMILAR_FILE_LATEST']}",
    f"NEWTODAY_FILE_LATEST: {CONFIG['NEWTODAY_FILE_LATEST']}",
    f"SUBMIT_DIR: {CONFIG['SUBMIT_DIR']}",
    f"TEMPLATE: {CONFIG['TEMPLATE_PATH']}",
    f"ANSWER: {CONFIG['ANSWER_PATH']}",
    f"TOTAL_STUDENTS: {total_cnt}",
    f"NEWLY_GRADED: {new_cnt}",
    f"TODAY_FILES(KST {now_kst().date()}): {len(df_today)}",
    f"TAG_AUDIT_LATEST: {'tag_audit_latest.csv'}",
    f"REQUIRED_CELLS: {CONFIG['REQUIRED_CELL_COUNT']}",
    f"OPTIONAL_CELLS: {CONFIG['OPTIONAL_CELL_COUNT']}",
    f"REQUIRED_CELL_INDEXES: {CONFIG['REQUIRED_CELL_INDEXES']}",
    f"OPTIONAL_CELL_INDEXES: {CONFIG['OPTIONAL_CELL_INDEXES']}",
    f"REQUIRED_CELL_MAP: {CONFIG['REQUIRED_CELL_MAP']}",
    f"OPTIONAL_CELL_MAP: {CONFIG['OPTIONAL_CELL_MAP']}",
    f"SCORE_RULE: {CONFIG['SCORE_RULE']}",
    f"TIMEZONE: {CONFIG['TIMEZONE']}",
    f"채점 제외(정답 출력 없음): 필수=[{excluded_req_str}], 연습=[{excluded_opt_str}]"
]

# 신규 ID
if new_cnt > 0:
    log_lines.append("NEW_IDS: " + ", ".join(sorted(new_ids)))

# 점수 통계 & 분포를 로그에 포함
log_lines += STATS_BLOCK

log_msg = "\n".join(log_lines) + "\n"
log_path = Path(OUT_DIR) / "autograde_run.log"
with open(log_path, "a", encoding="utf-8") as f:
    f.write(log_msg)

print("🗒️ Log appended to:", str(log_path))

# 화면 출력용도 함께 표시

print(
    "✅ 완료:", OUT_DIR,
    "\n- 실행시각:", kst_now_str,
    f"\n- 전체 채점 학생 수: {total_cnt}명",
    f"\n- 새롭게 추가 채점한 학생 수: {new_cnt}명",
    f"\n- 오늘 들어온 파일 수(KST {now_kst().date()}): {len(df_today)}건",
    "\n- 최신 요약:", Path(summary_latest).name,
    f"\n- 최신 유사도: {Path(similar_latest).name if pairs else 'N/A'}",
    "\n- 최신 Today:", Path(newtoday_latest).name,
    f"\n- 실행 산출물 폴더: executed/{RUN_TS}/",
    "\n- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트)",
    "\n".join(["\n📊 Score & Distribution Summary"] + STATS_BLOCK))


🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
Tagged template arleady exist!
🔹 Step 3/8: Reading tagged template and answer (label-based)...
🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...
🔹 Step 5/8: Collecting and grading submissions...
🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...
🔹 Step 7/8: Saving outputs...
🔹 Step 8/8: Logging summary...
🗒️ Log appended to: /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시/autograde_run.log
✅ 완료: /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시 
- 실행시각: 2025-11-04 16:14:33 KST 
- 전체 채점 학생 수: 112명 
- 새롭게 추가 채점한 학생 수: 0명 
- 오늘 들어온 파일 수(KST 2025-11-04): 0건 
- 최신 요약: summary_static_with_name_latest.csv 
- 최신 유사도: similar_pairs_latest.csv 
- 최신 Today: new_today_latest.csv 
- 실행 산출물 폴더: executed/20251104_160742/ 
- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트) 
📊 Score & Distribution Summary
SCORE STATS:
- mean=95.4, median=99.5, min=0.0
STATUS × OUTPUT_MATCH (counts):


In [ ]:
##deleted
#policy.py 분리
#io_utils.py 분리
#nb_utils.py 분리
#reports.py 분리

## comment out
# sessions.toml 분리
# label_tagging.py 분리

# ====== 2) 경로 설정 (여기만 바꾸세요) ======

#session = load_config ("autograder/config/sessions.toml", 9, "DEV")   # 일단 9차시 2024본
session = load_config ("autograder/config/sessions.toml", 9, "PROD")   # 일단 9차시 2025본
TEMPLATE_PATH = session["template_path"]
SOLUTION_PATH = session["solution_path"]
SUBMIT_DIR    = session["submit_dir"]
OUT_DIR       = session["out_dir"]

"""
TEMPLATE_PATH = "/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR24/MR_09_파이썬 기초_학번_성명.ipynb"
SOLUTION_PATH = "/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR24/MR_09_파이썬 기초_학번_성명_src.ipynb"
SUBMIT_DIR    = "/content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/9차시"
OUT_DIR       = "/content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시"

"""

# ====== 3) 라이브러리 임포트 & 공통 유틸 ======
print("🔹 Step 1/8: Importing libraries & initializing...")
import os, re, unicodedata
import nbformat, pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt



# --- 시간/타임존 유틸 ---

RUN_TS = now_kst().strftime("%Y%m%d_%H%M%S")
OUT = Path(OUT_DIR); OUT.mkdir(parents=True, exist_ok=True)
EXEC_DIR = OUT / "executed" / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)



os.makedirs(OUT_DIR, exist_ok=True)
tagged_template_path = str(Path(OUT_DIR) / "template_tagged.ipynb")
tag_audit_path = str(Path(OUT_DIR) / "tag_audit.csv")



# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, sol_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    solcell = sol_map.get(lab, {}).get("cell")
    sol_out = _cell_output_text(solcell) if solcell else ""
    expected_output = bool(sol_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

# ====== 4) 템플릿 라벨 기반 태깅(없으면 생성) ======
#STRICT_REQUIRED_FROM_SOLUTION = True  # 정답에 코드가 있으면 무조건 필수로 태깅
# MIN_CODE_LEN = 5                      # "실코드" 최소 길이 기준

print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(tagged_template_path):
    template_label_tagging(TEMPLATE_PATH, SOLUTION_PATH,
                           tagged_template_path, tag_audit_path)
else :
    print ("Tagged template arleady exist!")

"""
    tmpl_nb = nbformat.read(TEMPLATE_PATH, as_version=4)
    sol_nb  = nbformat.read(SOLUTION_PATH, as_version=4)

    # 1) 라벨 맵
    tmpl_lmap = _label_map(tmpl_nb)
    sol_lmap  = _label_map(sol_nb)

    # 2) '연습문제' 섹션 감지(옵션) — 마크다운 구간 기반
    optional_labels = set()
    in_opt = False
    for i, cell in enumerate(tmpl_nb.cells):
        if cell.cell_type == "markdown":
            tx = cell.source or ""
            if re.search(r"(연습\s*문제|연습문제|Optional|옵션)", tx, re.I):
                in_opt = True
            elif re.match(r"^#{1,6}\s", tx):
                in_opt = False
        elif cell.cell_type == "code":
            lab = _extract_label(cell.source or "")
            if in_opt and lab:
                optional_labels.add(lab)

    # 3) 필수 라벨 결정
    required_labels = set()
    for lab, info in tmpl_lmap.items():
        if lab in optional_labels:
            continue
        tcell = info["cell"]
        scell = sol_lmap.get(lab, {}).get("cell", None)

        tcode = _normalize_code(tcell.source or "")
        scode = _normalize_code(scell.source or "") if scell else ""

        placeholder = bool(re.search(
            r"(TODO|여기에\s*코드|fill\s*in|pass\s*$|\.\.\.|채우세요|작성하시오|구현)",
            tcell.source or "", re.I | re.M
        ))

        if STRICT_REQUIRED_FROM_SOLUTION and scell and len(scode) >= MIN_CODE_LEN:
            required_labels.add(lab); continue
        if scell and len(scode) >= MIN_CODE_LEN and (tcode != scode):
            required_labels.add(lab); continue
        if (placeholder or (len(tcode) < MIN_CODE_LEN and len(scode) >= MIN_CODE_LEN)):
            required_labels.add(lab)

    # 4) 태그 메타데이터 쓰기
    for i, cell in enumerate(tmpl_nb.cells):
        if cell.cell_type != "code":
            continue
        lab = _extract_label(cell.source or "")
        tags = set(cell.get("metadata", {}).get("tags", []))
        if lab:
            if lab in required_labels:
                tags.add("required")
            if lab in optional_labels:
                tags.add("optional_ex")
            cell.setdefault("metadata", {})["tags"] = list(tags)
    nbformat.write(tmpl_nb, tagged_template_path)

# --- 태깅 감사 리포트 생성 ---
tmpl_lmap = _label_map(nbformat.read(tagged_template_path, as_version=4))
sol_lmap  = _label_map(nbformat.read(SOLUTION_PATH, as_version=4))
audit_rows = []
for lab, info in tmpl_lmap.items():
    tcell = info["cell"]
    scell = sol_lmap.get(lab, {}).get("cell", None)
    tcode = _normalize_code(tcell.source or "")
    scode = _normalize_code(scell.source or "") if scell else ""
    placeholder = bool(re.search(
        r"(TODO|여기에\s*코드|fill\s*in|pass\s*$|\.\.\.|채우세요|작성하시오|구현)",
        tcell.source or "", re.I | re.M
    ))
    optional = ("optional_ex" in (tcell.get("metadata", {}).get("tags", [])))
    required = ("required"     in (tcell.get("metadata", {}).get("tags", [])))
    audit_rows.append({
        "label": lab,
        "in_template": True,
        "in_solution": bool(scell),
        "t_len": len(tcode),
        "s_len": len(scode),
        "equal_code": (tcode == scode) if scell else None,
        "placeholder": placeholder,
        "optional_detected": optional,
        "required_decided": required,
    })
"""
# ====== 5) 태그 읽기(라벨 기반) + 레퍼런스 NB 로드 ======
print("🔹 Step 3/8: Reading tagged template and solution (label-based)...")
## tmpl = nbformat.read(tagged_template_path, as_version=4)
## sol  = nbformat.read(SOLUTION_PATH, as_version=4)  # 출력 비교용
tmpl = nbformat.read(tagged_template_path, as_version=4)
sol  = nbformat.read(SOLUTION_PATH, as_version=4)

req_labels, opt_labels = set(), set()
req_idx, opt_idx = set(), set()

for i, c in enumerate(tmpl.cells):
    if c.cell_type != "code":
        continue
    tg = set(c.get("metadata", {}).get("tags", []))
    lab = _extract_label(c.source or "")
    if "required" in tg and lab:
        req_labels.add(lab); req_idx.add(i)
    if "optional_ex" in tg and lab:
        opt_labels.add(lab); opt_idx.add(i)

tmpl_lmap = _label_map(tmpl)
sol_lmap  = _label_map(sol)
REQ_INDEX_LABELS = _indexes_with_labels(tmpl, req_idx)
OPT_INDEX_LABELS = _indexes_with_labels(tmpl, opt_idx)
template_fp = _nb_fingerprint(tmpl)

# ====== 6) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 7) 제출물 처리 ======
print("🔹 Step 5/8: Collecting and grading submissions...")
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]

for path in paths:
    P=Path(path)
    if P.name in {Path(TEMPLATE_PATH).name, Path(SOLUTION_PATH).name, Path(tagged_template_path).name}:
        continue

    sid, name = extract_id_and_name(P)
    sid2name[sid] = name
    sid2path[sid] = path

    # 노트북 로드
    try:
        nb = nbformat.read(path, as_version=4)
    except Exception as e:
        rows.append([sid, name, P.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=P.name
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, P.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    sol_lmap = _label_map(sol)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        solcell = sol_lmap.get(lab, {}).get("cell")
        sol_out = _cell_output_text(solcell) if solcell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        if not outputs_equal(stu_out, sol_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산

    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)

    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, P.name, score, status, reason, output_match, feedback])

    try:
        if mtime_kst(path).date() == now_kst().date():
            today_rows_tmp.append([sid, name, P.name, score, status, reason, output_match, feedback])
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)

# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 8) 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성

EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)

summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status","reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")

# --- 태깅 감사 리포트 저장 (있을 때만) ---
"""
if audit_rows:
    try:
        audit_df = pd.DataFrame(audit_rows)
        audit_file = Path(OUT_DIR) / "tag_audit.csv"
        audit_df.to_csv(audit_file, index=False, encoding="utf-8-sig")
    except Exception as e:
        print("⚠️ tag audit save failed:", e)
"""





##202511031314

# 완료 메시지 + 로그
print("🔹 Step 8/8: Logging summary...")
kst_now_str = now_kst().strftime("%Y-%m-%d %H:%M:%S KST")
total_cnt = len(df)
prev_ids = prev_ids or set()
new_ids = set(df["student_id"].astype(str)) - prev_ids if prev_ids else set(df["student_id"].astype(str))
new_cnt = len(new_ids)

# 전체 채점 제외 라벨(정답 출력 없음) 한 줄 요약
excluded_req_str = ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) if EXCLUDED_REQ_ALL else "없음"
excluded_opt_str = ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) if EXCLUDED_OPT_ALL else "없음"

## 점수 통계 및 분포 산출
STATS_BLOCK = build_stats_block(df)

CONFIG = {
    "RUN_TS": RUN_TS,
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "SOLUTION_PATH": str(SOLUTION_PATH),
    "SUBMIT_DIR": str(SUBMIT_DIR),
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tmpl, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tmpl, opt_idx),
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",
    "TIMEZONE": "KST (UTC+9)",
}

log_lines = [
    f"=== Autograde Run @ {kst_now_str} ({CONFIG['RUN_TS']}) ===",
    f"OUT_DIR: {CONFIG['OUT_DIR']}",
    f"EXEC_DIR: {CONFIG['EXEC_DIR']}",
    f"SUMMARY_FILE_LATEST: {CONFIG['SUMMARY_FILE_LATEST']}",
    f"SIMILAR_FILE_LATEST: {CONFIG['SIMILAR_FILE_LATEST']}",
    f"NEWTODAY_FILE_LATEST: {CONFIG['NEWTODAY_FILE_LATEST']}",
    f"SUBMIT_DIR: {CONFIG['SUBMIT_DIR']}",
    f"TEMPLATE: {CONFIG['TEMPLATE_PATH']}",
    f"SOLUTION: {CONFIG['SOLUTION_PATH']}",
    f"TOTAL_STUDENTS: {total_cnt}",
    f"NEWLY_GRADED: {new_cnt}",
    f"TODAY_FILES(KST {now_kst().date()}): {len(df_today)}",
    f"TAG_AUDIT_LATEST: {'tag_audit_latest.csv'}",
    f"REQUIRED_CELLS: {CONFIG['REQUIRED_CELL_COUNT']}",
    f"OPTIONAL_CELLS: {CONFIG['OPTIONAL_CELL_COUNT']}",
    f"REQUIRED_CELL_INDEXES: {CONFIG['REQUIRED_CELL_INDEXES']}",
    f"OPTIONAL_CELL_INDEXES: {CONFIG['OPTIONAL_CELL_INDEXES']}",
    f"REQUIRED_CELL_MAP: {CONFIG['REQUIRED_CELL_MAP']}",
    f"OPTIONAL_CELL_MAP: {CONFIG['OPTIONAL_CELL_MAP']}",
    f"SCORE_RULE: {CONFIG['SCORE_RULE']}",
    f"TIMEZONE: {CONFIG['TIMEZONE']}",
    f"채점 제외(정답 출력 없음): 필수=[{excluded_req_str}], 연습=[{excluded_opt_str}]"
]

# 신규 ID
if new_cnt > 0:
    log_lines.append("NEW_IDS: " + ", ".join(sorted(new_ids)))

# 점수 통계 & 분포를 로그에 포함
log_lines += STATS_BLOCK

log_msg = "\n".join(log_lines) + "\n"
log_path = Path(OUT_DIR) / "autograde_run.log"
with open(log_path, "a", encoding="utf-8") as f:
    f.write(log_msg)

print("🗒️ Log appended to:", str(log_path))

# 화면 출력용도 함께 표시

print(
    "✅ 완료:", OUT_DIR,
    "\n- 실행시각:", kst_now_str,
    f"\n- 전체 채점 학생 수: {total_cnt}명",
    f"\n- 새롭게 추가 채점한 학생 수: {new_cnt}명",
    f"\n- 오늘 들어온 파일 수(KST {now_kst().date()}): {len(df_today)}건",
    "\n- 최신 요약:", Path(summary_latest).name,
    f"\n- 최신 유사도: {Path(similar_latest).name if pairs else 'N/A'}",
    "\n- 최신 Today:", Path(newtoday_latest).name,
    f"\n- 실행 산출물 폴더: executed/{RUN_TS}/",
    "\n- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트)",
    "\n".join(["\n📊 Score & Distribution Summary"] + STATS_BLOCK))


<>:101: SyntaxWarning: invalid escape sequence '\s'
<>:101: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-725553996.py:101: SyntaxWarning: invalid escape sequence '\s'
  if re.search(r"(연습\s*문제|연습문제|Optional|옵션)", tx, re.I):


🔹 Step 1/8: Importing libraries & initializing...
🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
🔹 Step 3/8: Reading tagged template and solution (label-based)...
🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...
🔹 Step 5/8: Collecting and grading submissions...
🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...
🔹 Step 7/8: Saving outputs...
🔹 Step 8/8: Logging summary...
🗒️ Log appended to: /content/drive/MyDrive/Colab Notebooks/52.KHCU_MR25/Autograde/9차시/autograde_run.log
✅ 완료: /content/drive/MyDrive/Colab Notebooks/52.KHCU_MR25/Autograde/9차시 
- 실행시각: 2025-11-04 01:22:11 KST 
- 전체 채점 학생 수: 11명 
- 새롭게 추가 채점한 학생 수: 11명 
- 오늘 들어온 파일 수(KST 2025-11-04): 0건 
- 최신 요약: summary_static_with_name_latest.csv 
- 최신 유사도: N/A 
- 최신 Today: new_today_latest.csv 
- 실행 산출물 폴더: executed/20251104_012200/ 
- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트) 
📊 Score & Distribution Summary
SCORE STATS:
- mean=99.8, median=100.0, min=98.5
STATUS × OUTPUT_MATCH (count

In [ ]:
##deleted
#policy.py 분리
#io_utils.py 분리


## comment out
#nb_utils.py 분리
#reports.py 분리

# ====== 2) 경로 설정 (여기만 바꾸세요) ======
TEMPLATE_PATH = "/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR24/MR_09_파이썬 기초_학번_성명.ipynb"
SOLUTION_PATH = "/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR24/MR_09_파이썬 기초_학번_성명_src.ipynb"
SUBMIT_DIR    = "/content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/9차시"
OUT_DIR       = "/content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시"

# ====== 3) 라이브러리 임포트 & 공통 유틸 ======
print("🔹 Step 1/8: Importing libraries & initializing...")
import os, re, unicodedata
import nbformat, pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt



# --- 시간/타임존 유틸 ---
# KST = dt.timezone(dt.timedelta(hours=9))
## def now_kst(): return dt.datetime.now(tz=KST)
"""
def mtime_kst(path_str: str):
    ts = os.path.getmtime(path_str)
    return dt.datetime.fromtimestamp(ts, tz=dt.timezone.utc).astimezone(KST)
"""
RUN_TS = now_kst().strftime("%Y%m%d_%H%M%S")

os.makedirs(OUT_DIR, exist_ok=True)
tagged_template_path = str(Path(OUT_DIR) / "template_tagged.ipynb")


"""
# --- 코드/라벨/출력 유틸 ---
## LABEL_PATTERN = re.compile(r'^\s*#\s*([0-9]+(?:\.[0-9]+)*)')

def _label_key_robust(label_str: str):
    ""
    '#1.4.3' 같은 라벨 문자열을 안전하게 숫자 리스트로 변환해 정렬 키로 사용.
    - '#', '#3.', '#1..2' 등 비정형도 처리
    - 숫자가 하나도 없으면 큰 값으로 밀기
    ""
    if not isinstance(label_str, str):
        return [999999]
    s = label_str.strip()
    # 맨 앞의 '#' 제거
    if s.startswith('#'):
        s = s[1:]
    # '.'로 나눈 뒤 숫자인 것만 취함
    parts = [p for p in s.split('.') if p.isdigit()]
    if not parts:
        # 그래도 없으면, 전체에서 숫자 추출
        parts = re.findall(r'\d+', s)
    try:
        return [int(x) for x in parts] if parts else [999999]
    except Exception:
        return [999999]


def _normalize_code(s: str) -> str:
    if not s: return ""
    lines = [ln for ln in s.splitlines() if not ln.strip().startswith("#")]
    return re.sub(r"\s+", " ", "\n".join(lines)).strip()

def _nb_fingerprint(nb):
    chunks=[]
    for c in nb.cells:
        if c.cell_type=="code":
            t=_normalize_code(c.source or "")
            if t: chunks.append(t)
    return " ".join(chunks)



def _sim(a,b): return SequenceMatcher(None, a or "", b or "").ratio()

def _extract_label(text: str):
    if not text: return None
    text = text.replace("\ufeff","")
    text_norm = unicodedata.normalize("NFKC", text)
    m = LABEL_PATTERN.match(text_norm.strip())
    return m.group(1) if m else None


def _label_map(nb):
  노트북의 code cell을 라벨(#1.2.3) 기준으로 매핑. 동일 라벨이 여럿이면 마지막(최종) 셀을 택한다.
    m = {}
    for i, c in enumerate(nb.cells):
        if c.cell_type != "code":
            continue
        lab = _extract_label(c.source or "")
        if lab:
            m[lab] = {"idx": i, "cell": c}
    return m

def _indexes_with_labels(nb, idx_set):
    items = []
    for i in sorted(idx_set):
        try:
            cell = nb.cells[i]
            lab = _extract_label(cell.source or "")
            items.append(f"{i}:{('#'+lab) if lab else '(no-label)'}")
        except Exception:
            items.append(f"{i}:(error)")
    return items

def _cell_output_text(cell):
    ""노트북 code cell의 출력(JSON)을 가능한 문자열로 정규화.""
    outs = cell.get("outputs", []) or []
    chunks = []
    for o in outs:
        ot = o.get("output_type")
        if ot in ("stream",):
            chunks.append(o.get("text", ""))
        elif ot in ("execute_result", "display_data"):
            data = o.get("data", {})
            if "text/plain" in data:
                chunks.append(data["text/plain"])
            elif "text/html" in data:
                chunks.append(re.sub(r"\s+", " ", data["text/html"]))
            elif "image/png" in data:
                chunks.append("[image/png]")
        elif ot in ("error",):
            chunks.append("ERROR:" + " ".join(o.get("traceback", [])))
    text = "\n".join(chunks)
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text
"""

# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, sol_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    solcell = sol_map.get(lab, {}).get("cell")
    sol_out = _cell_output_text(solcell) if solcell else ""
    expected_output = bool(sol_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

# ====== 4) 템플릿 라벨 기반 태깅(없으면 생성) ======
STRICT_REQUIRED_FROM_SOLUTION = True  # 정답에 코드가 있으면 무조건 필수로 태깅
MIN_CODE_LEN = 5                      # "실코드" 최소 길이 기준

print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(tagged_template_path):
    tmpl_nb = nbformat.read(TEMPLATE_PATH, as_version=4)
    sol_nb  = nbformat.read(SOLUTION_PATH, as_version=4)

    # 1) 라벨 맵
    tmpl_lmap = _label_map(tmpl_nb)
    sol_lmap  = _label_map(sol_nb)

    # 2) '연습문제' 섹션 감지(옵션) — 마크다운 구간 기반
    optional_labels = set()
    in_opt = False
    for i, cell in enumerate(tmpl_nb.cells):
        if cell.cell_type == "markdown":
            tx = cell.source or ""
            if re.search(r"(연습\s*문제|연습문제|Optional|옵션)", tx, re.I):
                in_opt = True
            elif re.match(r"^#{1,6}\s", tx):
                in_opt = False
        elif cell.cell_type == "code":
            lab = _extract_label(cell.source or "")
            if in_opt and lab:
                optional_labels.add(lab)

    # 3) 필수 라벨 결정
    required_labels = set()
    for lab, info in tmpl_lmap.items():
        if lab in optional_labels:
            continue
        tcell = info["cell"]
        scell = sol_lmap.get(lab, {}).get("cell", None)

        tcode = _normalize_code(tcell.source or "")
        scode = _normalize_code(scell.source or "") if scell else ""

        placeholder = bool(re.search(
            r"(TODO|여기에\s*코드|fill\s*in|pass\s*$|\.\.\.|채우세요|작성하시오|구현)",
            tcell.source or "", re.I | re.M
        ))

        if STRICT_REQUIRED_FROM_SOLUTION and scell and len(scode) >= MIN_CODE_LEN:
            required_labels.add(lab); continue
        if scell and len(scode) >= MIN_CODE_LEN and (tcode != scode):
            required_labels.add(lab); continue
        if (placeholder or (len(tcode) < MIN_CODE_LEN and len(scode) >= MIN_CODE_LEN)):
            required_labels.add(lab)

    # 4) 태그 메타데이터 쓰기
    for i, cell in enumerate(tmpl_nb.cells):
        if cell.cell_type != "code":
            continue
        lab = _extract_label(cell.source or "")
        tags = set(cell.get("metadata", {}).get("tags", []))
        if lab:
            if lab in required_labels:
                tags.add("required")
            if lab in optional_labels:
                tags.add("optional_ex")
            cell.setdefault("metadata", {})["tags"] = list(tags)
    nbformat.write(tmpl_nb, tagged_template_path)

# --- 태깅 감사 리포트 생성 ---
tmpl_lmap = _label_map(nbformat.read(tagged_template_path, as_version=4))
sol_lmap  = _label_map(nbformat.read(SOLUTION_PATH, as_version=4))
audit_rows = []
for lab, info in tmpl_lmap.items():
    tcell = info["cell"]
    scell = sol_lmap.get(lab, {}).get("cell", None)
    tcode = _normalize_code(tcell.source or "")
    scode = _normalize_code(scell.source or "") if scell else ""
    placeholder = bool(re.search(
        r"(TODO|여기에\s*코드|fill\s*in|pass\s*$|\.\.\.|채우세요|작성하시오|구현)",
        tcell.source or "", re.I | re.M
    ))
    optional = ("optional_ex" in (tcell.get("metadata", {}).get("tags", [])))
    required = ("required"     in (tcell.get("metadata", {}).get("tags", [])))
    audit_rows.append({
        "label": lab,
        "in_template": True,
        "in_solution": bool(scell),
        "t_len": len(tcode),
        "s_len": len(scode),
        "equal_code": (tcode == scode) if scell else None,
        "placeholder": placeholder,
        "optional_detected": optional,
        "required_decided": required,
    })

# ====== 5) 태그 읽기(라벨 기반) + 레퍼런스 NB 로드 ======
print("🔹 Step 3/8: Reading tagged template and solution (label-based)...")
tmpl = nbformat.read(tagged_template_path, as_version=4)
sol  = nbformat.read(SOLUTION_PATH, as_version=4)  # 출력 비교용

req_labels, opt_labels = set(), set()
req_idx, opt_idx = set(), set()

for i, c in enumerate(tmpl.cells):
    if c.cell_type != "code":
        continue
    tg = set(c.get("metadata", {}).get("tags", []))
    lab = _extract_label(c.source or "")
    if "required" in tg and lab:
        req_labels.add(lab); req_idx.add(i)
    if "optional_ex" in tg and lab:
        opt_labels.add(lab); opt_idx.add(i)

tmpl_lmap = _label_map(tmpl)
sol_lmap  = _label_map(sol)
REQ_INDEX_LABELS = _indexes_with_labels(tmpl, req_idx)
OPT_INDEX_LABELS = _indexes_with_labels(tmpl, opt_idx)
template_fp = _nb_fingerprint(tmpl)

# ====== 6) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 7) 제출물 처리 ======
print("🔹 Step 5/8: Collecting and grading submissions...")
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]

for path in paths:
    P=Path(path)
    if P.name in {Path(TEMPLATE_PATH).name, Path(SOLUTION_PATH).name, Path(tagged_template_path).name}:
        continue

    sid, name = extract_id_and_name(P)
    sid2name[sid] = name
    sid2path[sid] = path

    # 노트북 로드
    try:
        nb = nbformat.read(path, as_version=4)
    except Exception as e:
        rows.append([sid, name, P.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=P.name
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, P.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    sol_lmap = _label_map(sol)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        solcell = sol_lmap.get(lab, {}).get("cell")
        sol_out = _cell_output_text(solcell) if solcell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        if not outputs_equal(stu_out, sol_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산

    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)

    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, P.name, score, status, reason, output_match, feedback])

    try:
        if mtime_kst(path).date() == now_kst().date():
            today_rows_tmp.append([sid, name, P.name, score, status, reason, output_match, feedback])
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)

# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 8) 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성

EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)

summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status","reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")

# --- 태깅 감사 리포트 저장 (있을 때만) ---
try:
    audit_df = pd.DataFrame(audit_rows)
    audit_filename_ts = f"tag_audit_{RUN_TS}.csv"
    audit_latest = Path(OUT_DIR) / "tag_audit_latest.csv"
    (EXEC_DIR / audit_filename_ts).parent.mkdir(parents=True, exist_ok=True)
    audit_df.to_csv(EXEC_DIR / audit_filename_ts, index=False, encoding="utf-8-sig")
    audit_df.to_csv(audit_latest, index=False, encoding="utf-8-sig")
except Exception as e:
    print("⚠️ tag audit save failed:", e)


"""
# ====== (NEW) 성적 통계 & 도수 분포 ====== 202511031314

    # 점수 통계
    score_mean = float(df["score"].astype(float).mean()) if not df.empty else 0.0
    score_median = float(df["score"].astype(float).median()) if not df.empty else 0.0
    score_min = float(df["score"].astype(float).min()) if not df.empty else 0.0

    # status x output_match 도수 분포 (피벗)
    ct = (
        df.groupby(["status", "output_match"])
          .size()
          .reset_index(name="count")
          .pivot_table(index="status", columns="output_match", values="count", fill_value=0)
          .sort_index()
    )

    # 텍스트로 예쁘게 변환
    ct_lines = []
    if not ct.empty:
        # 열 정렬: ZERO/ERROR/MISSING/MISMATCH/OK 순서(존재하는 것만)
        col_order = [c for c in ["ZERO", "ERROR", "MISSING", "MISMATCH", "OK"] if c in ct.columns]
        ct = ct[col_order]
        header = " " * 14 + " | " + " | ".join([f"{c:>8}" for c in col_order])
        sep = "-" * len(header)
        ct_lines.append(header)
        ct_lines.append(sep)
        for idx, row in ct.iterrows():
            ct_lines.append(f"{str(idx):>14} | " + " | ".join([f"{int(row[c]):8d}" for c in col_order]))
    else:
        ct_lines.append("(no data)")

    # 로그 문자열 추가용
    STATS_BLOCK = [
        "SCORE STATS:",
        f"- mean={score_mean:.1f}, median={score_median:.1f}, min={score_min:.1f}",
        "STATUS × OUTPUT_MATCH (counts):",
        *ct_lines
    ]

    """
STATS_BLOCK = build_stats_block(df)

# 화면 출력용도 함께 표시
print("\n".join(["\n📊 Score & Distribution Summary"] + STATS_BLOCK))


##202511031314

# 완료 메시지 + 로그
print("🔹 Step 8/8: Logging summary...")
kst_now_str = now_kst().strftime("%Y-%m-%d %H:%M:%S KST")
total_cnt = len(df)
prev_ids = prev_ids or set()
new_ids = set(df["student_id"].astype(str)) - prev_ids if prev_ids else set(df["student_id"].astype(str))
new_cnt = len(new_ids)

# 전체 채점 제외 라벨(정답 출력 없음) 한 줄 요약
excluded_req_str = ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) if EXCLUDED_REQ_ALL else "없음"
excluded_opt_str = ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) if EXCLUDED_OPT_ALL else "없음"


CONFIG = {
    "RUN_TS": RUN_TS,
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "SOLUTION_PATH": str(SOLUTION_PATH),
    "SUBMIT_DIR": str(SUBMIT_DIR),
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tmpl, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tmpl, opt_idx),
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",
    "TIMEZONE": "KST (UTC+9)",
}

log_lines = [
    f"=== Autograde Run @ {kst_now_str} ({CONFIG['RUN_TS']}) ===",
    f"OUT_DIR: {CONFIG['OUT_DIR']}",
    f"EXEC_DIR: {CONFIG['EXEC_DIR']}",
    f"SUMMARY_FILE_LATEST: {CONFIG['SUMMARY_FILE_LATEST']}",
    f"SIMILAR_FILE_LATEST: {CONFIG['SIMILAR_FILE_LATEST']}",
    f"NEWTODAY_FILE_LATEST: {CONFIG['NEWTODAY_FILE_LATEST']}",
    f"SUBMIT_DIR: {CONFIG['SUBMIT_DIR']}",
    f"TEMPLATE: {CONFIG['TEMPLATE_PATH']}",
    f"SOLUTION: {CONFIG['SOLUTION_PATH']}",
    f"TOTAL_STUDENTS: {total_cnt}",
    f"NEWLY_GRADED: {new_cnt}",
    f"TODAY_FILES(KST {now_kst().date()}): {len(df_today)}",
    f"TAG_AUDIT_LATEST: {'tag_audit_latest.csv'}",
    f"REQUIRED_CELLS: {CONFIG['REQUIRED_CELL_COUNT']}",
    f"OPTIONAL_CELLS: {CONFIG['OPTIONAL_CELL_COUNT']}",
    f"REQUIRED_CELL_INDEXES: {CONFIG['REQUIRED_CELL_INDEXES']}",
    f"OPTIONAL_CELL_INDEXES: {CONFIG['OPTIONAL_CELL_INDEXES']}",
    f"REQUIRED_CELL_MAP: {CONFIG['REQUIRED_CELL_MAP']}",
    f"OPTIONAL_CELL_MAP: {CONFIG['OPTIONAL_CELL_MAP']}",
    f"SCORE_RULE: {CONFIG['SCORE_RULE']}",
    f"TIMEZONE: {CONFIG['TIMEZONE']}",
    f"채점 제외(정답 출력 없음): 필수=[{excluded_req_str}], 연습=[{excluded_opt_str}]",
]

# 점수 통계 & 분포를 로그에 포함
log_lines += STATS_BLOCK



if new_cnt > 0:
    log_lines.append("NEW_IDS: " + ", ".join(sorted(new_ids)))
log_msg = "\n".join(log_lines) + "\n"

print(
    "✅ 완료:", OUT_DIR,
    "\n- 실행시각:", kst_now_str,
    f"\n- 전체 채점 학생 수: {total_cnt}명",
    f"\n- 새롭게 추가 채점한 학생 수: {new_cnt}명",
    f"\n- 오늘 들어온 파일 수(KST {now_kst().date()}): {len(df_today)}건",
    "\n- 최신 요약:", Path(summary_latest).name,
    f"\n- 최신 유사도: {Path(similar_latest).name if pairs else 'N/A'}",
    "\n- 최신 Today:", Path(newtoday_latest).name,
    f"\n- 실행 산출물 폴더: executed/{RUN_TS}/",
    "\n- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트)"
)

log_path = Path(OUT_DIR) / "autograde_run.log"
with open(log_path, "a", encoding="utf-8") as f:
    f.write(log_msg)
print("🗒️ Log appended to:", str(log_path))


<>:42: SyntaxWarning: invalid escape sequence '\s'
<>:42: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-251540224.py:42: SyntaxWarning: invalid escape sequence '\s'
  ## LABEL_PATTERN = re.compile(r'^\s*#\s*([0-9]+(?:\.[0-9]+)*)')


🔹 Step 1/8: Importing libraries & initializing...
🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
🔹 Step 3/8: Reading tagged template and solution (label-based)...
🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...
🔹 Step 5/8: Collecting and grading submissions...
🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...
🔹 Step 7/8: Saving outputs...

📊 Score & Distribution Summary
SCORE STATS:
- mean=95.4, median=99.5, min=0.0
STATUS × OUTPUT_MATCH (counts):
               |     ZERO |    ERROR |  MISSING | MISMATCH |       OK
---------------------------------------------------------------------
         ERROR |        0 |        1 |        0 |        0 |        0
    INCOMPLETE |        0 |        0 |       14 |       42 |        0
            OK |        0 |        0 |        0 |        0 |       53
          ZERO |        2 |        0 |        0 |        0 |        0
🔹 Step 8/8: Logging summary...
✅ 완료: /content/drive/MyDrive/Colab Notebooks/20

In [ ]:
#policy.py 분리
#io_utils.py 분리

# ====== 2) 경로 설정 (여기만 바꾸세요) ======
TEMPLATE_PATH = "/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR24/MR_09_파이썬 기초_학번_성명.ipynb"
SOLUTION_PATH = "/content/drive/MyDrive/Colab Notebooks/52.KHCU_MR24/MR_09_파이썬 기초_학번_성명_src.ipynb"
SUBMIT_DIR    = "/content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/9차시"
OUT_DIR       = "/content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시"

# ====== 3) 라이브러리 임포트 & 공통 유틸 ======
print("🔹 Step 1/8: Importing libraries & initializing...")
import os, re, unicodedata
import nbformat, pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt



# --- 시간/타임존 유틸 ---
KST = dt.timezone(dt.timedelta(hours=9))
## def now_kst(): return dt.datetime.now(tz=KST)
def mtime_kst(path_str: str):
    ts = os.path.getmtime(path_str)
    return dt.datetime.fromtimestamp(ts, tz=dt.timezone.utc).astimezone(KST)
RUN_TS = now_kst().strftime("%Y%m%d_%H%M%S")

os.makedirs(OUT_DIR, exist_ok=True)
tagged_template_path = str(Path(OUT_DIR) / "template_tagged.ipynb")

# --- 코드/라벨/출력 유틸 ---
LABEL_PATTERN = re.compile(r'^\s*#\s*([0-9]+(?:\.[0-9]+)*)')

def _label_key_robust(label_str: str):
    """
    '#1.4.3' 같은 라벨 문자열을 안전하게 숫자 리스트로 변환해 정렬 키로 사용.
    - '#', '#3.', '#1..2' 등 비정형도 처리
    - 숫자가 하나도 없으면 큰 값으로 밀기
    """
    if not isinstance(label_str, str):
        return [999999]
    s = label_str.strip()
    # 맨 앞의 '#' 제거
    if s.startswith('#'):
        s = s[1:]
    # '.'로 나눈 뒤 숫자인 것만 취함
    parts = [p for p in s.split('.') if p.isdigit()]
    if not parts:
        # 그래도 없으면, 전체에서 숫자 추출
        parts = re.findall(r'\d+', s)
    try:
        return [int(x) for x in parts] if parts else [999999]
    except Exception:
        return [999999]


def _normalize_code(s: str) -> str:
    if not s: return ""
    lines = [ln for ln in s.splitlines() if not ln.strip().startswith("#")]
    return re.sub(r"\s+", " ", "\n".join(lines)).strip()

def _nb_fingerprint(nb):
    chunks=[]
    for c in nb.cells:
        if c.cell_type=="code":
            t=_normalize_code(c.source or "")
            if t: chunks.append(t)
    return " ".join(chunks)

def _sim(a,b): return SequenceMatcher(None, a or "", b or "").ratio()

def _extract_label(text: str):
    if not text: return None
    text = text.replace("\ufeff","")
    text_norm = unicodedata.normalize("NFKC", text)
    m = LABEL_PATTERN.match(text_norm.strip())
    return m.group(1) if m else None

def _label_map(nb):
    """노트북의 code cell을 라벨(#1.2.3) 기준으로 매핑. 동일 라벨이 여럿이면 마지막(최종) 셀을 택한다."""
    m = {}
    for i, c in enumerate(nb.cells):
        if c.cell_type != "code":
            continue
        lab = _extract_label(c.source or "")
        if lab:
            m[lab] = {"idx": i, "cell": c}
    return m

def _indexes_with_labels(nb, idx_set):
    items = []
    for i in sorted(idx_set):
        try:
            cell = nb.cells[i]
            lab = _extract_label(cell.source or "")
            items.append(f"{i}:{('#'+lab) if lab else '(no-label)'}")
        except Exception:
            items.append(f"{i}:(error)")
    return items

def _cell_output_text(cell):
    """노트북 code cell의 출력(JSON)을 가능한 문자열로 정규화."""
    outs = cell.get("outputs", []) or []
    chunks = []
    for o in outs:
        ot = o.get("output_type")
        if ot in ("stream",):
            chunks.append(o.get("text", ""))
        elif ot in ("execute_result", "display_data"):
            data = o.get("data", {})
            if "text/plain" in data:
                chunks.append(data["text/plain"])
            elif "text/html" in data:
                chunks.append(re.sub(r"\s+", " ", data["text/html"]))
            elif "image/png" in data:
                chunks.append("[image/png]")
        elif ot in ("error",):
            chunks.append("ERROR:" + " ".join(o.get("traceback", [])))
    text = "\n".join(chunks)
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text
"""
def _outputs_equal_loose_text(a: str, b: str) -> bool:
    ""숫자는 완전일치(오차 허용 X). 공백/개행/연속공백은 무시.""
    def strip_space(s: str) -> str:
        # 연속 공백/개행/탭 → 단일 공백으로 축소
        return re.sub(r"\s+", " ", (s or "").strip())
    a_s = strip_space(a)
    b_s = strip_space(b)
    # 숫자 토큰 단위 완전 일치 확인
    num_pat = r"[-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?"
    a_nums = re.findall(num_pat, a_s)
    b_nums = re.findall(num_pat, b_s)
    if len(a_nums) != len(b_nums):
        return False
    if any(x != y for x, y in zip(a_nums, b_nums)):
        return False
    # 숫자 외 텍스트는 공백 무시 후 일치로 판단
    def remove_nums(s):
        return re.sub(num_pat, "", s)
    return strip_space(remove_nums(a_s)) == strip_space(remove_nums(b_s))
    """

"""
def extract_id_and_name(p: Path):
    ""한글 안전: NFC 정규화 + 다양한 구분자 허용 + 노트북 내부 fallback.""
    name = ""
    sid = None
    fname = unicodedata.normalize("NFC", p.name)
    stem  = unicodedata.normalize("NFC", p.stem)

    m = re.search(r"(\d{7,})[ _-]+(.+)\.ipynb$", fname, re.IGNORECASE)
    if m:
        sid = m.group(1)
        name = m.group(2)
        name = re.sub(r"[_ ]?\((\d+)\)$", "", name)
        name = re.sub(r"[_ ]?（(\d+)）$", "", name)  # 전각
        name = name.strip().replace("_"," ")
        name = re.sub(r"\s{2,}", " ", name)
    else:
        m2 = re.search(r"(\d{7,})", stem)
        if m2:
            sid = m2.group(1)

    if not name:
        try:
            nb_ = nbformat.read(str(p), as_version=4)
            for c in nb_.cells:
                if c.cell_type == "code":
                    src = unicodedata.normalize("NFC", c.source or "")
                    m3 = re.search(r"(?:NAME|STUDENT_NAME|학생명|이름|성명)\s*=\s*['\"]([^'\"]+)['\"]", src, re.I)
                    if m3: name = m3.group(1).strip(); break
            if not name:
                for c in nb_.cells:
                    if c.cell_type == "markdown":
                        txt = unicodedata.normalize("NFC", c.source or "")
                        m4 = re.search(r"(?:이름|성명)\s*[:：]\s*([^\n]+)", txt)
                        if m4: name = m4.group(1).strip(); break
            if not name:
                md = getattr(nb_, "metadata", {}) or {}
                for key in ("student_name","name","성명","이름"):
                    if key in md and str(md[key]).strip():
                        name = str(md[key]).strip(); break
        except Exception:
            pass

    if not sid:  sid = f"UNIDENTIFIED_{stem}"
    if not name: name = "이름미상"
    name = unicodedata.normalize("NFC", name)
    return sid, name

# 제출파일 정보 추출
def _filesize_bytes(path: str) -> int:
    try:
        return os.path.getsize(path)
    except Exception:
        return -1

def _mtime_kst_str(path: str) -> str:
    try:
        ts = os.path.getmtime(path)
        return dt.datetime.fromtimestamp(ts, tz=dt.timezone.utc).astimezone(KST).strftime("%Y-%m-%d %H:%M:%S KST")
    except Exception:
        return ""
        """

# ====== 정답 출력 있는 셀만 채점 ======
def _exec_and_out_expected(stu_map, sol_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생: 실제 출력(stu_out)이 있어야 '실행'으로 인정
    - expected_output=False → 채점 대상에서 제외(정보만 반환)
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    solcell = sol_map.get(lab, {}).get("cell")
    sol_out = _cell_output_text(solcell) if solcell else ""
    expected_output = bool(sol_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)

    # 채점 대상인 경우(정답 출력 있음) → '출력 길이 > 0' 이어야 실행 인정
    executed = bool(stu_out) if expected_output else (bool(stu_out) or (scell.get("execution_count") not in (None, 0)))
    return executed, stu_out, expected_output

# ====== 4) 템플릿 라벨 기반 태깅(없으면 생성) ======
STRICT_REQUIRED_FROM_SOLUTION = True  # 정답에 코드가 있으면 무조건 필수로 태깅
MIN_CODE_LEN = 5                      # "실코드" 최소 길이 기준

print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")
if not os.path.exists(tagged_template_path):
    tmpl_nb = nbformat.read(TEMPLATE_PATH, as_version=4)
    sol_nb  = nbformat.read(SOLUTION_PATH, as_version=4)

    # 1) 라벨 맵
    tmpl_lmap = _label_map(tmpl_nb)
    sol_lmap  = _label_map(sol_nb)

    # 2) '연습문제' 섹션 감지(옵션) — 마크다운 구간 기반
    optional_labels = set()
    in_opt = False
    for i, cell in enumerate(tmpl_nb.cells):
        if cell.cell_type == "markdown":
            tx = cell.source or ""
            if re.search(r"(연습\s*문제|연습문제|Optional|옵션)", tx, re.I):
                in_opt = True
            elif re.match(r"^#{1,6}\s", tx):
                in_opt = False
        elif cell.cell_type == "code":
            lab = _extract_label(cell.source or "")
            if in_opt and lab:
                optional_labels.add(lab)

    # 3) 필수 라벨 결정
    required_labels = set()
    for lab, info in tmpl_lmap.items():
        if lab in optional_labels:
            continue
        tcell = info["cell"]
        scell = sol_lmap.get(lab, {}).get("cell", None)

        tcode = _normalize_code(tcell.source or "")
        scode = _normalize_code(scell.source or "") if scell else ""

        placeholder = bool(re.search(
            r"(TODO|여기에\s*코드|fill\s*in|pass\s*$|\.\.\.|채우세요|작성하시오|구현)",
            tcell.source or "", re.I | re.M
        ))

        if STRICT_REQUIRED_FROM_SOLUTION and scell and len(scode) >= MIN_CODE_LEN:
            required_labels.add(lab); continue
        if scell and len(scode) >= MIN_CODE_LEN and (tcode != scode):
            required_labels.add(lab); continue
        if (placeholder or (len(tcode) < MIN_CODE_LEN and len(scode) >= MIN_CODE_LEN)):
            required_labels.add(lab)

    # 4) 태그 메타데이터 쓰기
    for i, cell in enumerate(tmpl_nb.cells):
        if cell.cell_type != "code":
            continue
        lab = _extract_label(cell.source or "")
        tags = set(cell.get("metadata", {}).get("tags", []))
        if lab:
            if lab in required_labels:
                tags.add("required")
            if lab in optional_labels:
                tags.add("optional_ex")
            cell.setdefault("metadata", {})["tags"] = list(tags)
    nbformat.write(tmpl_nb, tagged_template_path)

# --- 태깅 감사 리포트 생성 ---
tmpl_lmap = _label_map(nbformat.read(tagged_template_path, as_version=4))
sol_lmap  = _label_map(nbformat.read(SOLUTION_PATH, as_version=4))
audit_rows = []
for lab, info in tmpl_lmap.items():
    tcell = info["cell"]
    scell = sol_lmap.get(lab, {}).get("cell", None)
    tcode = _normalize_code(tcell.source or "")
    scode = _normalize_code(scell.source or "") if scell else ""
    placeholder = bool(re.search(
        r"(TODO|여기에\s*코드|fill\s*in|pass\s*$|\.\.\.|채우세요|작성하시오|구현)",
        tcell.source or "", re.I | re.M
    ))
    optional = ("optional_ex" in (tcell.get("metadata", {}).get("tags", [])))
    required = ("required"     in (tcell.get("metadata", {}).get("tags", [])))
    audit_rows.append({
        "label": lab,
        "in_template": True,
        "in_solution": bool(scell),
        "t_len": len(tcode),
        "s_len": len(scode),
        "equal_code": (tcode == scode) if scell else None,
        "placeholder": placeholder,
        "optional_detected": optional,
        "required_decided": required,
    })

# ====== 5) 태그 읽기(라벨 기반) + 레퍼런스 NB 로드 ======
print("🔹 Step 3/8: Reading tagged template and solution (label-based)...")
tmpl = nbformat.read(tagged_template_path, as_version=4)
sol  = nbformat.read(SOLUTION_PATH, as_version=4)  # 출력 비교용

req_labels, opt_labels = set(), set()
req_idx, opt_idx = set(), set()

for i, c in enumerate(tmpl.cells):
    if c.cell_type != "code":
        continue
    tg = set(c.get("metadata", {}).get("tags", []))
    lab = _extract_label(c.source or "")
    if "required" in tg and lab:
        req_labels.add(lab); req_idx.add(i)
    if "optional_ex" in tg and lab:
        opt_labels.add(lab); opt_idx.add(i)

tmpl_lmap = _label_map(tmpl)
sol_lmap  = _label_map(sol)
REQ_INDEX_LABELS = _indexes_with_labels(tmpl, req_idx)
OPT_INDEX_LABELS = _indexes_with_labels(tmpl, opt_idx)
template_fp = _nb_fingerprint(tmpl)

# ====== 6) 옵션/임계값 & 이전 결과 로드 ======
print("🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")
TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD     = 0.99

prev_ids = set()
prev_files = sorted(
    [p for p in Path(OUT_DIR).glob("summary_static_with_name_*.csv") if p.is_file()],
    key=lambda p: os.path.getmtime(p)
)
if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ====== 7) 제출물 처리 ======
print("🔹 Step 5/8: Collecting and grading submissions...")
paths = sorted([str(p) for p in Path(SUBMIT_DIR).glob("**/*.ipynb")])
rows=[]; fps={}
sid2file={}; sid2name={}; sid2path={}
today_rows_tmp=[]

# 전체(전역) 채점 제외 라벨 수집(학생 피드백엔 미노출)
EXCLUDED_REQ_ALL = set()
EXCLUDED_OPT_ALL = set()

def _label_key(x):
    try: return [int(t) for t in x.split('.')]
    except: return [999999]

for path in paths:
    P=Path(path)
    if P.name in {Path(TEMPLATE_PATH).name, Path(SOLUTION_PATH).name, Path(tagged_template_path).name}:
        continue

    sid, name = extract_id_and_name(P)
    sid2name[sid] = name
    sid2path[sid] = path

    # 노트북 로드
    try:
        nb = nbformat.read(path, as_version=4)
    except Exception as e:
        rows.append([sid, name, P.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 템플릿 유사도(원본 제출 0점)
    fp=_nb_fingerprint(nb); fps[sid]=fp; sid2file[sid]=P.name
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, P.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])
        try:
            if mtime_kst(path).date() == now_kst().date():
                today_rows_tmp.append(rows[-1])
        except Exception: pass
        continue

    # 학생 라벨 맵
    stu_lmap = _label_map(nb)
    sol_lmap = _label_map(sol)

    # === 채점: 정답 '출력 있음' 라벨만 평가 ===
   # base_score = 100.0

    req_missing = []          # 필수 미실행 라벨(정답 출력 있는 라벨만)
    req_mismatch = []         # 필수 출력 불일치 라벨(정답 출력 있는 라벨만)
    opt_missing = []          # 옵션 미실행 라벨(정답 출력 있는 라벨만)

    excluded_req = []         # 이번 파일에서 채점 제외된 필수 라벨
    excluded_opt = []         # 이번 파일에서 채점 제외된 연습 라벨

    # 1) 필수 라벨
    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        solcell = sol_lmap.get(lab, {}).get("cell")
        sol_out = _cell_output_text(solcell) if solcell else ""
        if not executed:
            req_missing.append(f"#{lab}"); continue
        #if not _outputs_equal_loose_text(stu_out, sol_out):
        if not outputs_equal(stu_out, sol_out):
            req_mismatch.append(f"#{lab}")

    # 2) 옵션(연습) 라벨
    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, sol_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")
        # 연습 오답은 감점 없음 (보고용/무시)

    # 감점 계산
    """
    score = base_score
    score -= 1.0  * len(req_missing)    # 필수 미실행
    score -= 0.5  * len(req_mismatch)   # 필수 실행-오답
    score -= 0.2  * len(opt_missing)    # 연습 미실행
    score = float(max(0.0, min(100.0, round(score, 1))))
    """
    score = BASE_SCORE
    score -= PENALTY_REQUIRED_MISS    * len(req_missing)
    score -= PENALTY_REQUIRED_MISMATCH * len(req_mismatch)
    score -= PENALTY_OPTIONAL_MISS     * len(opt_missing)


    # 상태/피드백 및 output_match 규칙
    fbl = []
    if req_missing:
        fbl.append(f"[필수 미실행 {len(req_missing)}개] 셀: " + ", ".join(req_missing))
    if req_mismatch:
        fbl.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: " + ", ".join(req_mismatch))
    if opt_missing:
        fbl.append(f"[연습문제 미실행 {len(opt_missing)}개] 셀: " + ", ".join(opt_missing))

    """
    status = "OK" if (not req_missing and not req_mismatch) else "INCOMPLETE"
    reason = "" if status=="OK" else "필수 미실행/불일치 존재"

    if status == "INCOMPLETE":
        output_match = "MISMATCH" if req_mismatch else "MISSING"
    else:
        output_match = "OK"
    """
    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"


    # 채점 내역 한 줄 요약 추가
    pen_required_missing  = len(req_missing)
    pen_required_mismatch = len(req_mismatch)
    pen_optional_missing  = len(opt_missing)
    """
    score_line = (
        f"(채점) base=100"
        f"{' − 1.0×' + str(pen_required_missing)  + '(필수 미실행)' if pen_required_missing  else ''}"
        f"{' − 0.5×' + str(pen_required_mismatch) + '(필수 불일치)' if pen_required_mismatch else ''}"
        f"{' − 0.2×' + str(pen_optional_missing)  + '(연습 미실행)' if pen_optional_missing  else ''}"
        f" = {score}"
    )
    """
    score_line = (
    f"(채점) base={BASE_SCORE}"
    f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing  else ''}"
    f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
    f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing  else ''}"
    f" = {score}"
)

    feedback = ("\n".join(fbl)) if fbl else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = (feedback + ("\n" if feedback else "") + score_line).strip()

    rows.append([sid, name, P.name, score, status, reason, output_match, feedback])

    try:
        if mtime_kst(path).date() == now_kst().date():
            today_rows_tmp.append([sid, name, P.name, score, status, reason, output_match, feedback])
    except Exception:
        pass

    # ── 전역 집합에 '채점 제외 라벨(정답 출력 없음)' 축적 (학생 피드백엔 미노출)
    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)

# 유사도 페어(≥0.99) + 학생명/저장일자/파일크기
print("🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
pairs=[]
sids=list(fps.keys())
for i in range(len(sids)):
    for j in range(i+1,len(sids)):
        a,b=sids[i], sids[j]
        sab=_sim(fps[a], fps[b])
        if sab>=0.99:
            file_a = sid2file.get(a, "")
            file_b = sid2file.get(b, "")
            path_a = sid2path.get(a, "")
            path_b = sid2path.get(b, "")
            name_a = sid2name.get(a, "")
            name_b = sid2name.get(b, "")
            mtime_a = _mtime_kst_str(path_a) if path_a else ""
            mtime_b = _mtime_kst_str(path_b) if path_b else ""
            size_a  = _filesize_bytes(path_a) if path_a else -1
            size_b  = _filesize_bytes(path_b) if path_b else -1
            pairs.append([
                a, name_a, file_a, mtime_a, size_a,
                b, name_b, file_b, mtime_b, size_b,
                f"{sab:.3f}"
            ])

# ====== 8) 저장 & 로그 ======
print("🔹 Step 7/8: Saving outputs...")


# -- 로그 파일 정보 생성

EXEC_ROOT = Path(OUT_DIR) / "executed"
EXEC_DIR  = EXEC_ROOT / RUN_TS
EXEC_DIR.mkdir(parents=True, exist_ok=True)

summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

df = pd.DataFrame(
    rows,
    columns=["student_id","student_name","file","score","status","reasons","output_match","feedback"]
)

summary_path_ts = EXEC_DIR / summary_filename_ts
df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a","name_a","file_a","mtime_a_kst","size_a_bytes",
            "student_b","name_b","file_b","mtime_b_kst","size_b_bytes",
            "similarity"
        ]
    )
    similar_path_ts = EXEC_DIR / similar_filename_ts
    df_sim.to_csv(similar_path_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=df.columns)
newtoday_path_ts = EXEC_DIR / newtoday_filename_ts
df_today.to_csv(newtoday_path_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")

# --- 태깅 감사 리포트 저장 (있을 때만) ---
try:
    audit_df = pd.DataFrame(audit_rows)
    audit_filename_ts = f"tag_audit_{RUN_TS}.csv"
    audit_latest = Path(OUT_DIR) / "tag_audit_latest.csv"
    (EXEC_DIR / audit_filename_ts).parent.mkdir(parents=True, exist_ok=True)
    audit_df.to_csv(EXEC_DIR / audit_filename_ts, index=False, encoding="utf-8-sig")
    audit_df.to_csv(audit_latest, index=False, encoding="utf-8-sig")
except Exception as e:
    print("⚠️ tag audit save failed:", e)

# ====== (NEW) 성적 통계 & 도수 분포 ====== 202511031314
try:
    # 점수 통계
    score_mean = float(df["score"].astype(float).mean()) if not df.empty else 0.0
    score_median = float(df["score"].astype(float).median()) if not df.empty else 0.0
    score_min = float(df["score"].astype(float).min()) if not df.empty else 0.0

    # status x output_match 도수 분포 (피벗)
    ct = (
        df.groupby(["status", "output_match"])
          .size()
          .reset_index(name="count")
          .pivot_table(index="status", columns="output_match", values="count", fill_value=0)
          .sort_index()
    )

    # 텍스트로 예쁘게 변환
    ct_lines = []
    if not ct.empty:
        # 열 정렬: ZERO/ERROR/MISSING/MISMATCH/OK 순서(존재하는 것만)
        col_order = [c for c in ["ZERO", "ERROR", "MISSING", "MISMATCH", "OK"] if c in ct.columns]
        ct = ct[col_order]
        header = " " * 14 + " | " + " | ".join([f"{c:>8}" for c in col_order])
        sep = "-" * len(header)
        ct_lines.append(header)
        ct_lines.append(sep)
        for idx, row in ct.iterrows():
            ct_lines.append(f"{str(idx):>14} | " + " | ".join([f"{int(row[c]):8d}" for c in col_order]))
    else:
        ct_lines.append("(no data)")

    # 로그 문자열 추가용
    STATS_BLOCK = [
        "SCORE STATS:",
        f"- mean={score_mean:.1f}, median={score_median:.1f}, min={score_min:.1f}",
        "STATUS × OUTPUT_MATCH (counts):",
        *ct_lines
    ]
    # 화면 출력용도 함께 표시
    print("\n".join(["\n📊 Score & Distribution Summary"] + STATS_BLOCK))

except Exception as _e_stats:
    STATS_BLOCK = [f"SCORE STATS: error: {_e_stats}"]
    print("⚠️ Stats block failed:", _e_stats)
##202511031314

# 완료 메시지 + 로그
print("🔹 Step 8/8: Logging summary...")
kst_now_str = now_kst().strftime("%Y-%m-%d %H:%M:%S KST")
total_cnt = len(df)
prev_ids = prev_ids or set()
new_ids = set(df["student_id"].astype(str)) - prev_ids if prev_ids else set(df["student_id"].astype(str))
new_cnt = len(new_ids)

# 전체 채점 제외 라벨(정답 출력 없음) 한 줄 요약
excluded_req_str = ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) if EXCLUDED_REQ_ALL else "없음"
excluded_opt_str = ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) if EXCLUDED_OPT_ALL else "없음"


CONFIG = {
    "RUN_TS": RUN_TS,
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "SOLUTION_PATH": str(SOLUTION_PATH),
    "SUBMIT_DIR": str(SUBMIT_DIR),
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tmpl, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tmpl, opt_idx),
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",
    "TIMEZONE": "KST (UTC+9)",
}

log_lines = [
    f"=== Autograde Run @ {kst_now_str} ({CONFIG['RUN_TS']}) ===",
    f"OUT_DIR: {CONFIG['OUT_DIR']}",
    f"EXEC_DIR: {CONFIG['EXEC_DIR']}",
    f"SUMMARY_FILE_LATEST: {CONFIG['SUMMARY_FILE_LATEST']}",
    f"SIMILAR_FILE_LATEST: {CONFIG['SIMILAR_FILE_LATEST']}",
    f"NEWTODAY_FILE_LATEST: {CONFIG['NEWTODAY_FILE_LATEST']}",
    f"SUBMIT_DIR: {CONFIG['SUBMIT_DIR']}",
    f"TEMPLATE: {CONFIG['TEMPLATE_PATH']}",
    f"SOLUTION: {CONFIG['SOLUTION_PATH']}",
    f"TOTAL_STUDENTS: {total_cnt}",
    f"NEWLY_GRADED: {new_cnt}",
    f"TODAY_FILES(KST {now_kst().date()}): {len(df_today)}",
    f"TAG_AUDIT_LATEST: {'tag_audit_latest.csv'}",
    f"REQUIRED_CELLS: {CONFIG['REQUIRED_CELL_COUNT']}",
    f"OPTIONAL_CELLS: {CONFIG['OPTIONAL_CELL_COUNT']}",
    f"REQUIRED_CELL_INDEXES: {CONFIG['REQUIRED_CELL_INDEXES']}",
    f"OPTIONAL_CELL_INDEXES: {CONFIG['OPTIONAL_CELL_INDEXES']}",
    f"REQUIRED_CELL_MAP: {CONFIG['REQUIRED_CELL_MAP']}",
    f"OPTIONAL_CELL_MAP: {CONFIG['OPTIONAL_CELL_MAP']}",
    f"SCORE_RULE: {CONFIG['SCORE_RULE']}",
    f"TIMEZONE: {CONFIG['TIMEZONE']}",
    f"채점 제외(정답 출력 없음): 필수=[{excluded_req_str}], 연습=[{excluded_opt_str}]",
]

# 점수 통계 & 분포를 로그에 포함
log_lines += STATS_BLOCK



if new_cnt > 0:
    log_lines.append("NEW_IDS: " + ", ".join(sorted(new_ids)))
log_msg = "\n".join(log_lines) + "\n"

print(
    "✅ 완료:", OUT_DIR,
    "\n- 실행시각:", kst_now_str,
    f"\n- 전체 채점 학생 수: {total_cnt}명",
    f"\n- 새롭게 추가 채점한 학생 수: {new_cnt}명",
    f"\n- 오늘 들어온 파일 수(KST {now_kst().date()}): {len(df_today)}건",
    "\n- 최신 요약:", Path(summary_latest).name,
    f"\n- 최신 유사도: {Path(similar_latest).name if pairs else 'N/A'}",
    "\n- 최신 Today:", Path(newtoday_latest).name,
    f"\n- 실행 산출물 폴더: executed/{RUN_TS}/",
    "\n- 템플릿 태깅본: template_tagged.ipynb (OUT_DIR 루트)"
)

log_path = Path(OUT_DIR) / "autograde_run.log"
with open(log_path, "a", encoding="utf-8") as f:
    f.write(log_msg)
print("🗒️ Log appended to:", str(log_path))


<>:125: SyntaxWarning: invalid escape sequence '\s'
<>:150: SyntaxWarning: invalid escape sequence '\d'
<>:125: SyntaxWarning: invalid escape sequence '\s'
<>:150: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1894651980.py:125: SyntaxWarning: invalid escape sequence '\s'
  return re.sub(r"\s+", " ", (s or "").strip())
/tmp/ipython-input-1894651980.py:150: SyntaxWarning: invalid escape sequence '\d'
  m = re.search(r"(\d{7,})[ _-]+(.+)\.ipynb$", fname, re.IGNORECASE)


🔹 Step 1/8: Importing libraries & initializing...
🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
🔹 Step 3/8: Reading tagged template and solution (label-based)...
🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...
🔹 Step 5/8: Collecting and grading submissions...
🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...
🔹 Step 7/8: Saving outputs...

📊 Score & Distribution Summary
SCORE STATS:
- mean=95.4, median=99.5, min=0.0
STATUS × OUTPUT_MATCH (counts):
               |     ZERO |    ERROR |  MISSING | MISMATCH |       OK
---------------------------------------------------------------------
         ERROR |        0 |        1 |        0 |        0 |        0
    INCOMPLETE |        0 |        0 |       14 |       42 |        0
            OK |        0 |        0 |        0 |        0 |       53
          ZERO |        2 |        0 |        0 |        0 |        0
🔹 Step 8/8: Logging summary...
✅ 완료: /content/drive/MyDrive/Colab Notebooks/20